In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents. For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

A helpful resource for getting up to speed with vector representations of documents is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [1]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
from nltk import word_tokenize
from nltk.util import ngrams

In [2]:
# read in the movie review corpus
def readReviews():
    raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/cornell_reviews.json").text.strip()
    corpus = [json.loads(line) for line in raw.split("\n")]
    return corpus

This is where you will implement two functions to featurize the data.

In [54]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer, FeatureHasher
from sklearn.feature_extraction.text import TfidfVectorizer
import re
dictvectorizer = DictVectorizer()
tfidfvectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
#tfidfvectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english', ngram_range=(1,2))

def tokens(doc):
    """Extract tokens from doc.
    """
    return (tok.lower() for tok in re.findall(r"\w+", doc))

def token_freqs(doc):
    """Extract a dict mapping tokens from doc to their frequencies."""
    freq = defaultdict(int)
    for tok in tokens(doc):
        freq[tok] += 1
    return freq

def createBasicFeatures(corpus):
    #Your code here
    classes = []
    vocab = []
    all_text = []
    for i in corpus:
        classes.append(i.get('class'))
        text = i.get('text')
        temp = token_freqs(text)
        vocab =  list(sorted(set(vocab + list(temp.keys()))))
        all_text.append(temp)
  
    texts = dictvectorizer.fit_transform(all_text) 
    return texts,classes,vocab

def bigram_freqs(doc):
    """Extract a dict mapping tokens from doc to their frequencies."""
    freq = defaultdict(int)
    for tok in doc:
        freq[tok] += 1
    return freq

def token_text(doc):
    """Extract a dict mapping tokens from doc to their frequencies."""
    text = []
    for tok in tokens(doc):
        text.append(tok)
    return text

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.

'''Bigram implementation takes very long time'''
"""
def createFancyFeatures(corpus):
  #Your code here
    classes = []
    vocab = []
    all_text = []
    for i in corpus:
        classes.append(i.get('class'))
        text = i.get('text')
        token = word_tokenize(text)
        bigram = list(ngrams(token, 2)) 
        temp = bigram_freqs(bigram)
        vocab =  list(sorted(set(vocab + list(temp.keys()))))
        all_text.append(temp)
  
    texts = vectorizer.fit_transform(all_text) 
    return texts,classes,vocab"""

def createFancyFeatures(corpus):
    #Your code here
    classes = []
    vocab = []
    all_text = []
    for i in corpus:
        classes.append(i.get('class'))
        text = i.get('text')
        all_text.append(text)  
        
    texts = tfidfvectorizer.fit_transform(all_text) 
    vocab = tfidfvectorizer.get_feature_names()
    return texts,classes,vocab

In [7]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

Run the following to train and evaluate two models using basic features:

In [9]:
corpus = readReviews()

In [11]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.825500
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'masterpiece', 'enjoyable', 'using', 'excellent', 'sherri', 'edge', '7', 'overall', 'master', 'fun', 'follows', 'perfectly', 'gas', 'different', 'quite', 'works', 'solid']
The most informative terms for neg are: ['waste', 'mess', 'lame', 'ridiculous', 'awful', 'worst', 'unfortunately', 'headed', 'boring', 'cheap', '0', 'write', 'superior', 'bad', 'tedious', 'terrible', 'maybe', 'supposed', 'jesse', 'poor']
----------L2 Norm-----------
The model's average accuracy is 0.834000
The most informative terms for pos are: ['fun', 'great', 'back', 'excellent', 'quite', 'well', 'seen', 'overall', 'perfectly', 'american', 'yet', 'memorable', 'pulp', 'job', 'terrific', 'true', 'very', 'different', 'performances', 'masterpiece']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'boring', 'only', 'script', 'plot',

Run the following to train and evaluate two models using extended features:

In [55]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.753500
The most informative terms for pos are: ['great', 'life', 'perfectly', 'perfect', 'memorable', 'quite', 'excellent', 'seen', 'hilarious', 'performances', 'different', 'performance', 'true', 'overall', 'war', 'terrific', 'best', 'world', 'jackie', 'especially']
The most informative terms for neg are: ['bad', 'worst', 'boring', 'supposed', 'waste', 'ridiculous', 'mess', 'awful', 'stupid', 'wasted', 'plot', 'script', 'reason', 'dull', 'lame', 'unfortunately', 'looks', 'attempt', 'fails', 'maybe']
----------L2 Norm-----------
The model's average accuracy is 0.861500
The most informative terms for pos are: ['great', 'life', 'perfect', 'hilarious', 'excellent', 'perfectly', 'memorable', 'terrific', 'true', 'best', 'quite', 'performances', 'world', 'different', 'performance', 'fun', 'family', 'especially', 'overall', 'wonderful']
The most informative terms for neg are: ['bad', 'worst', 'boring', 'plot', 'supposed', 'stupid'